In [2]:
# Import necessary libraries and replace 'C:\WinUtils' with the full relative path to your WinUtils folder

import os, sys

os.environ["HADOOP_HOME"] = "C:\WinUtils"
os.environ["PYSPARK_PYTHON"] = "python.exe"
os.environ["PATH"] = os.getenv("path") + ";" + os.getenv("HADOOP_HOME") + "\\bin"

In [3]:
# Initialise Spark session

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Transaction Dataset Map Reduce script") \
    .getOrCreate()

In [4]:
# Replace 'Your_Dataset.csv' with the full relative path to your dataset and read its data


data = spark.read.option("Header", True).option("InferSchema", True).csv("Your_Dataset.csv")
data.printSchema()

root
 |-- Type: string (nullable = true)
 |-- Days for shipping (real): integer (nullable = true)
 |-- Days for shipment (scheduled): integer (nullable = true)
 |-- Benefit per order: double (nullable = true)
 |-- Sales per customer: double (nullable = true)
 |-- Delivery Status: string (nullable = true)
 |-- Late_delivery_risk: integer (nullable = true)
 |-- Category Id: integer (nullable = true)
 |-- Category Name: string (nullable = true)
 |-- Customer City: string (nullable = true)
 |-- Customer Country: string (nullable = true)
 |-- Customer Email: string (nullable = true)
 |-- Customer Fname: string (nullable = true)
 |-- Customer Id: integer (nullable = true)
 |-- Customer Lname: string (nullable = true)
 |-- Customer Password: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Customer State: string (nullable = true)
 |-- Customer Street: string (nullable = true)
 |-- Customer Zipcode: integer (nullable = true)
 |-- Department Id: integer (nullable = 

In [5]:
#Show data

data.show()

+--------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+-----------+--------------+--------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+--------------------+---------------+-------------+---------------+-------------------+-------------------+--------------------+------------+-------------+--------------+--------------------------+--------------+
|    Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sale

In [6]:
# Map data

mapped_data = data.rdd.map(lambda row: (row[8], 1))
mapped_data.take(7) #This will be showing the first 7 results

[('Sporting Goods', 1),
 ('Sporting Goods', 1),
 ('Sporting Goods', 1),
 ('Sporting Goods', 1),
 ('Sporting Goods', 1),
 ('Sporting Goods', 1),
 ('Sporting Goods', 1)]

In [7]:
# Reduce data

reduced_data = mapped_data.reduceByKey(lambda a,b: a+b)

# Sort data

sorted_data = reduced_data.sortBy(lambda x: -x[1])
sorted_data.collect()

[('Cleats', 24551),
 ("Men's Footwear", 22246),
 ("Women's Apparel", 21035),
 ('Indoor/Outdoor Games', 19298),
 ('Fishing', 17325),
 ('Water Sports', 15540),
 ('Camping & Hiking', 13729),
 ('Cardio Equipment', 12487),
 ('Shop By Sport', 10984),
 ('Electronics', 3156),
 ('Accessories', 1780),
 ('Golf Balls', 1475),
 ("Girls' Apparel", 1201),
 ('Golf Gloves', 1070),
 ('Trade-In', 974),
 ('Video Games', 838),
 ("Children's Clothing", 652),
 ("Women's Clothing", 650),
 ('Baseball & Softball', 632),
 ('Hockey', 614),
 ('Cameras ', 592),
 ('Toys', 529),
 ('Golf Shoes', 524),
 ('Pet Supplies', 492),
 ('Crafts', 484),
 ('Garden', 484),
 ('DVDs', 483),
 ('Computers', 442),
 ('Golf Apparel', 441),
 ('Hunting & Shooting', 440),
 ('Music', 434),
 ('Consumer Electronics', 431),
 ('Boxing & MMA', 423),
 ('Books ', 405),
 ("Kids' Golf Clubs", 384),
 ('Health and Beauty', 362),
 ('Sporting Goods', 357),
 ('Lacrosse', 343),
 ('Tennis & Racquet', 328),
 ('Fitness Accessories', 309),
 ("Men's Golf Clubs"

In [8]:
# Convert to DataFrame

result = sorted_data.toDF(['Category', 'Count'])
result.show()

+--------------------+-----+
|            Category|Count|
+--------------------+-----+
|              Cleats|24551|
|      Men's Footwear|22246|
|     Women's Apparel|21035|
|Indoor/Outdoor Games|19298|
|             Fishing|17325|
|        Water Sports|15540|
|    Camping & Hiking|13729|
|    Cardio Equipment|12487|
|       Shop By Sport|10984|
|         Electronics| 3156|
|         Accessories| 1780|
|          Golf Balls| 1475|
|      Girls' Apparel| 1201|
|         Golf Gloves| 1070|
|            Trade-In|  974|
|         Video Games|  838|
| Children's Clothing|  652|
|    Women's Clothing|  650|
| Baseball & Softball|  632|
|              Hockey|  614|
+--------------------+-----+
only showing top 20 rows



In [1]:
# Save result as CSV


result.coalesce(1) \
   .write \
   .option("header", "true") \
   .csv("result")

NameError: name 'result' is not defined

In [13]:
# Convert to Pandas DataFrame

pandasDf = result.toPandas()
pandasDf

,Category,Count
0,Cleats,24551
1,Men's Footwear,22246
2,Women's Apparel,21035
3,Indoor/Outdoor Games,19298
4,Fishing,17325
5,Water Sports,15540
6,Camping & Hiking,13729
7,Cardio Equipment,12487
8,Shop By Sport,10984
9,Electronics,3156


In [11]:
# Save Pandas DataFrame as CSV
# Replace 'Output_TransactionMapReduceInfo.csv' with the name you want for your output file

pandasDf.to_csv('Output_TransactionMapReduceInfo.csv')